In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

def assign_attributes(df):
    # 初期フレーム抽出
    initial = df[df['frame'] == 1].copy()
    n_points = len(initial)
    # ペア数の決定: 点数//2 の最大3、最小1
    if n_points < 2:
        df['assigned_role'] = 'nan_nan'
        return df
    n_pairs = min(3, n_points // 2)
    n_pairs = max(1, n_pairs)

    # 初期フレーム上で距離行列を計算
    coords = initial[['x', 'y']].values
    dist_matrix = cdist(coords, coords)
    np.fill_diagonal(dist_matrix, np.inf)

    # 最も近いペアをn_pairs組取得
    pairs = []
    tmp = dist_matrix.copy()
    for _ in range(n_pairs):
        idx = np.argmin(tmp)
        i, j = divmod(idx, tmp.shape[1])
        pairs.append((initial.iloc[i], initial.iloc[j]))
        tmp[i, :] = np.inf
        tmp[:, i] = np.inf
        tmp[j, :] = np.inf
        tmp[:, j] = np.inf

    # ゴール位置(0,752.5)を基準にD/Oを決定
    reference = np.array([0, 752.5])
    pair_roles = []
    for p1, p2 in pairs:
        d1 = np.linalg.norm(p1[['x', 'y']].values - reference)
        d2 = np.linalg.norm(p2[['x', 'y']].values - reference)
        if d1 < d2:
            pair_roles.append((p1, 'D', p2, 'O'))
        else:
            pair_roles.append((p2, 'D', p1, 'O'))

    # yの中点でソートし、right/top/leftを割り当て
    positions = ['right', 'top', 'left'][:n_pairs]
    mid_list = sorted(
        [((p1['y'] + p2['y']) / 2, p1, rD, p2, rO)
         for (p1, rD, p2, rO) in pair_roles],
        key=lambda x: x[0], reverse=True
    )

    # ID→役割マッピングの作成
    role_map = {}
    for idx, (_, pD, rD, pO, rO) in enumerate(mid_list):
        pos = positions[idx]
        role_map[pD['ID']] = f"{rD}_{pos}"
        role_map[pO['ID']] = f"{rO}_{pos}"

    # 全フレームにマッピングを適用（nan_nanは発生しない）
    df['assigned_role'] = df['ID'].map(role_map)
    return df

In [ ]:
input_dir = "../../output/CAMELTrack_outputs/Indoor/transformed/"
output_dir = "../../output/CAMELTrack_outputs/Indoor/transformed_with_attributes"
os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.endswith(".txt"):
        input_path = os.path.join(input_dir, file_name)
        df = pd.read_csv(input_path, header=None, names=["frame", "ID", "x", "y"])
        result_df = assign_attributes(df)
        
        output_path = os.path.join(output_dir, file_name)
        result_df.to_csv(output_path, index=False, header=False)
        print(f"Processed file saved to {output_path}")

Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T1_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T2_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T3_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T4_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T5_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T6_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S1T7_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S2T1_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transformed_with_attributes/basket_S2T2_pre.txt
Processed file saved to ../../CAMELTrack_outputs/Indoor/transfor